In [2]:
#VGG 19 Model and results

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import urllib
import io
import skimage.transform
from gen_adver import FastGradient

import theano
import theano.tensor as T
from theano import pp

import lasagne
from lasagne.layers import InputLayer, DenseLayer, DropoutLayer
from lasagne.layers.conv import Conv2DLayer as ConvLayer
from lasagne.layers.pool import Pool2DLayer as PoolLayer
from lasagne.layers import NonlinearityLayer
from lasagne.layers import DropoutLayer
from lasagne.layers import LocalResponseNormalization2DLayer as NormLayer
from lasagne.utils import floatX
from lasagne.nonlinearities import softmax

In [13]:
def prep_image(url,mean_image):
    ext = url.split('.')[-1]
    im = plt.imread(io.BytesIO(urllib.urlopen(url).read()), ext)
    # Resize so smallest dim = 256, preserving aspect ratio
    h, w, _ = im.shape
    if h < w:
        im = skimage.transform.resize(im, (256, w*256/h), preserve_range=True)
    else:
        im = skimage.transform.resize(im, (h*256/w, 256), preserve_range=True)

    # Central crop to 224x224
    h, w, _ = im.shape
    im = im[h//2-112:h//2+112, w//2-112:w//2+112]
    rawim = np.copy(im).astype('uint8')
    
    # Shuffle axes to c01
    im = np.swapaxes(np.swapaxes(im, 1, 2), 0, 1)
    
    # Convert to BGR
    im = im[::-1, :, :]
    im = im - mean_image[:,None,None]
    return rawim, floatX(im[np.newaxis])

In [14]:
# Model to be trained/tested against

def build_model(input_var,batch_size = None):
    net = {}
    net['input'] = InputLayer((batch_size, 3, 224, 224),input_var=input_var)
    net['conv1_1'] = ConvLayer(
        net['input'], 64, 3, pad=1)
    net['conv1_2'] = ConvLayer(
        net['conv1_1'], 64, 3, pad=1)
    net['pool1'] = PoolLayer(net['conv1_2'], 2)
    net['conv2_1'] = ConvLayer(
        net['pool1'], 128, 3, pad=1)
    net['conv2_2'] = ConvLayer(
        net['conv2_1'], 128, 3, pad=1)
    net['pool2'] = PoolLayer(net['conv2_2'], 2)
    net['conv3_1'] = ConvLayer(
        net['pool2'], 256, 3, pad=1)
    net['conv3_2'] = ConvLayer(
        net['conv3_1'], 256, 3, pad=1)
    net['conv3_3'] = ConvLayer(
        net['conv3_2'], 256, 3, pad=1)
    net['conv3_4'] = ConvLayer(
        net['conv3_3'], 256, 3, pad=1)
    net['pool3'] = PoolLayer(net['conv3_4'], 2)
    net['conv4_1'] = ConvLayer(
        net['pool3'], 512, 3, pad=1)
    net['conv4_2'] = ConvLayer(
        net['conv4_1'], 512, 3, pad=1)
    net['conv4_3'] = ConvLayer(
        net['conv4_2'], 512, 3, pad=1)
    net['conv4_4'] = ConvLayer(
        net['conv4_3'], 512, 3, pad=1)
    net['pool4'] = PoolLayer(net['conv4_4'], 2)
    net['conv5_1'] = ConvLayer(
        net['pool4'], 512, 3, pad=1)
    net['conv5_2'] = ConvLayer(
        net['conv5_1'], 512, 3, pad=1)
    net['conv5_3'] = ConvLayer(
        net['conv5_2'], 512, 3, pad=1)
    net['conv5_4'] = ConvLayer(
        net['conv5_3'], 512, 3, pad=1)
    net['pool5'] = PoolLayer(net['conv5_4'], 2)
    net['fc6'] = DenseLayer(net['pool5'], num_units=4096)
    net['fc6_dropout'] = DropoutLayer(net['fc6'], p=0.5)
    net['fc7'] = DenseLayer(net['fc6_dropout'], num_units=4096)
    net['fc7_dropout'] = DropoutLayer(net['fc7'], p=0.5)
    net['fc8'] = DenseLayer(
        net['fc7_dropout'], num_units=1000, nonlinearity=None)
    net['prob'] = NonlinearityLayer(net['fc8'], softmax)

    return net

In [15]:
# Load model parameters and metadata
def load_data():
    model = pickle.load(open('/afs/ir/users/m/e/meric/Documents/CS231N/CS231N-FinalProject/datasets/vgg19.pkl'))

    classes = model['synset words']
    mean_image= model['mean value']
    
    return model, classes, mean_image

In [29]:
def download_val_images (num_ex, mean_image,gold_labels):
    ''' 
    Dynamically downloads sample images from ImageNet.  
    '''
    index = urllib.urlopen('http://www.image-net.org/challenges/LSVRC/2012/ori_urls/indexval.html').read()
    image_urls = index.split('<br>')
    final_labels = gold_labels.copy()
    np.random.seed(37)
    np.random.shuffle(image_urls)
    np.random.seed(37)
    np.random.shuffle(final_labels)
    result_labels = np.zeros(num_ex)

    images = np.zeros ((num_ex, 3, 224, 224), dtype=np.float32)
    i = 0
    j = 0
    used=True
    rawim = np.zeros ((num_ex, 224, 224, 3), dtype=np.float32)
    for im_url in image_urls:
        print i, im_url
        # only use quick downloads on flickr
        if 'static.flickr' not in im_url:
            continue
            
        rawim[i,:,:,:], result = prep_image (im_url, mean_image)
      
        if result.any():
            images[i,:,:,:] = result
            result_labels[i] = final_labels[j]
            i += 1
           
        if i >= num_ex: 
            break
        j += 1
            
    print images.shape
    return rawim,images,result_labels
    

In [34]:
def load_gold_labels():
    f = open("../datasets/val_gold_labels.txt",'r')
    labels = []
    for line in f:
        labels.append(int(line))
        
    return np.asarray(labels)

In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
def find_adverserial_examples(tot_images=1,batch_size=1):
    
    cls = FastGradient(num_images = batch_size,input_dim=(3,224,224), eps=0.7, 
                   loss='softmax')
    model, classes, mean_image = load_data()
    gold_labels = load_gold_labels()
    rawim, images,gold_labels = download_val_images (tot_images, mean_image, gold_labels)
    input_var = T.tensor4('inputs')
    net = build_model(input_var,batch_size=batch_size)
    lasagne.layers.set_all_param_values(net['prob'], model['param values'])

    if batch_size > tot_images:
        print "Input the correct batch size and the total images in the dataset"
        exit(1)

    for i in xrange(tot_images/batch_size):
        curSet = images[i*batch_size:(i+1)*batch_size,:,:,:]
        true_prob = np.array(lasagne.layers.get_output(net['prob'], curSet, deterministic=True).eval())
        true_top5 = np.argsort(true_prob,axis=1)[:,-1:-6:-1]

        newim = rawim[i*batch_size:(i+1)*batch_size,:,:,:].transpose(0,3,1,2)
        final = cls.adExample(newim,np.array(gold_labels[i*batch_size:(i+1)*batch_size]),model['param values'],net['prob'],input_var)
        final = final - mean_image[None,:,None,None]
        adv_prob = np.array(lasagne.layers.get_output(net['prob'], final, deterministic=True).eval())
        adv_top5 = np.argsort(adv_prob,axis=1)[:,-1:-6:-1]
        final = final + mean_image[None,:,None,None]

        for k in xrange(batch_size):
            plt.subplot(121)
            plt.imshow(rawim[k,:,:,:].astype('uint8'))
            for n, label in enumerate(true_top5[k,:]):
                plt.text(0,260,'Original Image')
                plt.text(0, 280 + n * 20, '{}. {} {} %'.format(n+1, classes[label],true_prob[k,label]*100), fontsize=12)

            plt.subplot(122)
            plt.imshow(final[k,:,:,:].transpose(1,2,0).astype('uint8'))
            for n, label in enumerate(adv_top5[k,:]):
                plt.text(0,260,'Adverserial Image')
                plt.text(240, 280 + n * 20, '{}. {} {} %'.format(n+1, classes[label],adv_prob[k,label]*100), fontsize=12)

            plt.show()


In [37]:
%matplotlib inline

def run_fsg_adverserial(tot_images=1,batch_size=1):
    find_adverserial_examples(tot_images=1,batch_size=1)
    

run_fsg_adverserial(tot_images=1,batch_size=1)
    

IOError: [Errno 2] No such file or directory: '../datasets/val_gold_labels.txt'